# Fantasy premier league data adventures

In [50]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
import json
import PremierLeague
from ast import literal_eval

In [51]:
pd.set_option('display.max_rows', None)

In [52]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Getting data

In [53]:
pl_table = PremierLeague.get_league_table()

In [54]:
pl_table['Last 5']=pl_table['Last 5'].astype('|S').str.decode("utf-8")
pl_table

,Squad,MP,W,D,L,GF,GA,GD,Pts,Pts/MP,xG,xGA,xGD,xGD/90,Last 5
Rk,,,,,,,,,,,,,,,
1,Arsenal,14,12,1,1,33,11,22,37,2.64,26.2,11.8,14.3,1.02,W D W W W
2,Manchester City,14,10,2,2,40,14,26,32,2.29,27.6,11.2,16.4,1.17,L W W W L
3,Newcastle Utd,15,8,6,1,29,11,18,30,2.00,24.3,14.3,9.9,0.66,W W W W W
4,Tottenham,15,9,2,4,31,21,10,29,1.93,24.1,16.5,7.6,0.50,L L W L W
5,Manchester Utd,14,8,2,4,20,20,0,26,1.86,18.8,17.3,1.5,0.11,W D W L W
6,Liverpool,14,6,4,4,28,17,11,22,1.57,24.2,19.5,4.7,0.34,W L L W W
7,Brighton,14,6,3,5,23,19,4,21,1.50,21.5,15.9,5.6,0.40,D L W W L
8,Chelsea,14,6,3,5,17,17,0,21,1.50,15.4,19.3,-3.9,-0.28,D D L L L
9,Fulham,15,5,4,6,24,26,-2,19,1.27,19.6,27.3,-7.6,-0.51,W W D L L


In [55]:
def quantify_form(row):
    results = row['Last 5'].split(' ')
    # TODO: add directional bias to form
    form = 0
    form += results.count('W')*3
    form -= results.count('L')*3
    return form
    

In [56]:
pl_table['Form'] = pl_table.apply(quantify_form, axis=1)

In [57]:
pl_table.sort_values('Form', ascending=False)

,Squad,MP,W,D,L,GF,GA,GD,Pts,Pts/MP,xG,xGA,xGD,xGD/90,Last 5,Form
Rk,,,,,,,,,,,,,,,,
3,Newcastle Utd,15,8,6,1,29,11,18,30,2.00,24.3,14.3,9.9,0.66,W W W W W,15
1,Arsenal,14,12,1,1,33,11,22,37,2.64,26.2,11.8,14.3,1.02,W D W W W,12
13,Leicester City,15,5,2,8,25,25,0,17,1.13,17.6,22.2,-4.6,-0.31,W W L W W,9
5,Manchester Utd,14,8,2,4,20,20,0,26,1.86,18.8,17.3,1.5,0.11,W D W L W,6
2,Manchester City,14,10,2,2,40,14,26,32,2.29,27.6,11.2,16.4,1.17,L W W W L,3
18,Nott'ham Forest,15,3,4,8,11,30,-19,13,0.87,15.3,24.1,-8.8,-0.59,D W L D W,3
12,Aston Villa,15,5,3,7,16,22,-6,18,1.20,17.1,19.3,-2.2,-0.14,L W L W W,3
11,Crystal Palace,14,5,4,5,15,18,-3,19,1.36,14.0,18.3,-4.3,-0.31,W L W W L,3
6,Liverpool,14,6,4,4,28,17,11,22,1.57,24.2,19.5,4.7,0.34,W L L W W,3


In [58]:
LEAGUE_ID = 1342688
url = f"https://fantasy.premierleague.com/api/bootstrap-static/"
res = requests.get(url)
#print(json.dumps(res.json(), indent=4))


In [59]:
data = res.json()

In [60]:
elements_df = pd.DataFrame(data['elements'])
elements_types_df = pd.DataFrame(data['element_types'])
teams_df = pd.DataFrame(data['teams'])

In [61]:
elements_df.columns

Index(['chance_of_playing_next_round', 'chance_of_playing_this_round', 'code',
       'cost_change_event', 'cost_change_event_fall', 'cost_change_start',
       'cost_change_start_fall', 'dreamteam_count', 'element_type', 'ep_next',
       'ep_this', 'event_points', 'first_name', 'form', 'id', 'in_dreamteam',
       'news', 'news_added', 'now_cost', 'photo', 'points_per_game',
       'second_name', 'selected_by_percent', 'special', 'squad_number',
       'status', 'team', 'team_code', 'total_points', 'transfers_in',
       'transfers_in_event', 'transfers_out', 'transfers_out_event',
       'value_form', 'value_season', 'web_name', 'minutes', 'goals_scored',
       'assists', 'clean_sheets', 'goals_conceded', 'own_goals',
       'penalties_saved', 'penalties_missed', 'yellow_cards', 'red_cards',
       'saves', 'bonus', 'bps', 'influence', 'creativity', 'threat',
       'ict_index', 'starts', 'expected_goals', 'expected_assists',
       'expected_goal_involvements', 'expected_goals_con

In [62]:
df = elements_df[['id', 'web_name','team','element_type','selected_by_percent','now_cost','minutes','value_season','total_points', 'form','goals_scored',
       'assists', 'clean_sheets', 'goals_conceded', 'own_goals',
       'penalties_saved', 'penalties_missed', 'yellow_cards', 'red_cards',
       'saves', 'bonus', 'bps', 'influence', 'creativity', 'threat',
       'ict_index', 'starts', 'expected_goals', 'expected_assists',
       'expected_goal_involvements', 'expected_goals_conceded',
       'influence_rank', 'influence_rank_type', 'creativity_rank',
       'creativity_rank_type', 'threat_rank', 'threat_rank_type',
       'ict_index_rank', 'ict_index_rank_type',
       'corners_and_indirect_freekicks_order',
       'corners_and_indirect_freekicks_text', 'direct_freekicks_order',
       'direct_freekicks_text', 'penalties_order', 'penalties_text',
       'expected_goals_per_90', 'saves_per_90', 'expected_assists_per_90',
       'expected_goal_involvements_per_90', 'expected_goals_conceded_per_90',
       'goals_conceded_per_90', 'now_cost_rank', 'now_cost_rank_type',
       'form_rank', 'form_rank_type', 'points_per_game_rank',
       'points_per_game_rank_type', 'selected_rank', 'selected_rank_type',
       'starts_per_90', 'clean_sheets_per_90']]

In [63]:
%%capture
df['position'] = df.element_type.map(elements_types_df.set_index('id').singular_name)
df['team'] = df.team.map(teams_df.set_index('id').name)
df['value'] = df.value_season.astype(float)
df['form'] = df.form.astype(float)
df.drop(['value_season', 'element_type'], axis=1, inplace=True)
df['form_per_cost'] = df.form / df.now_cost
#df['']

df = df.loc[df.value > 0]

In [ ]:
player_id = 4
url = f'https://fantasy.premierleague.com/api/element-summary/{player_id}/'
res = requests.get(url)
d = res.json()
print(json.dumps(d, indent=4))

# Teams

In [69]:
team_pivot = df.pivot_table(index=['team','position'],values=['value','form','form_per_cost'],aggfunc=np.mean).reset_index()
team_pivot = team_pivot[['team', 'position', 'value', 'form', 'form_per_cost']]
team_pivot.loc[team_pivot.position == 'Goalkeeper',:].sort_values(['value', 'form'],ascending=False).head(10)
team_pivot.loc[team_pivot.position == 'Defender',:].sort_values(['value', 'form'],ascending=False).head(10)
team_pivot.loc[team_pivot.position == 'Midfielder',:].sort_values(['value', 'form'],ascending=False).head(10)
team_pivot.loc[team_pivot.position == 'Forward',:].sort_values(['value', 'form'],ascending=False).head(10)

,team,position,value,form,form_per_cost
42,Leicester,Goalkeeper,14.6,6.7,0.163415
62,Nott'm Forest,Goalkeeper,14.3,2.3,0.048936
14,Brentford,Goalkeeper,13.3,2.7,0.058696
58,Newcastle,Goalkeeper,13.2,4.7,0.088679
78,Wolves,Goalkeeper,12.2,1.7,0.034000
2,Arsenal,Goalkeeper,11.8,6.0,0.122449
46,Liverpool,Goalkeeper,11.8,2.7,0.049091
26,Crystal Palace,Goalkeeper,10.7,4.3,0.095556
50,Man City,Goalkeeper,10.6,4.3,0.079630
18,Brighton,Goalkeeper,10.4,2.7,0.058696


,team,position,value,form,form_per_cost
0,Arsenal,Defender,7.437500,3.800000,0.076718
40,Leicester,Defender,6.657143,2.871429,0.064968
56,Newcastle,Defender,6.060000,2.570000,0.051134
68,Spurs,Defender,5.580000,1.270000,0.025981
20,Chelsea,Defender,5.050000,0.525000,0.010145
52,Man Utd,Defender,4.850000,1.750000,0.037817
24,Crystal Palace,Defender,4.742857,2.028571,0.045296
48,Man City,Defender,4.688889,1.444444,0.026178
16,Brighton,Defender,4.671429,1.000000,0.022179
76,Wolves,Defender,4.487500,0.737500,0.017007


,team,position,value,form,form_per_cost
59,Newcastle,Midfielder,6.850000,3.170000,0.060240
19,Brighton,Midfielder,6.490000,2.860000,0.053117
39,Leeds,Midfielder,6.022222,2.777778,0.053639
3,Arsenal,Midfielder,5.491667,3.008333,0.052083
7,Aston Villa,Midfielder,5.333333,2.111111,0.042401
51,Man City,Midfielder,5.255556,2.300000,0.028080
43,Leicester,Midfielder,5.200000,2.450000,0.040405
15,Brentford,Midfielder,4.900000,1.420000,0.028852
27,Crystal Palace,Midfielder,4.780000,2.030000,0.036653
35,Fulham,Midfielder,4.730769,1.469231,0.028866


,team,position,value,form,form_per_cost
9,Bournemouth,Forward,8.800000,4.500000,0.081098
49,Man City,Forward,7.700000,3.000000,0.041530
13,Brentford,Forward,6.633333,2.800000,0.042436
69,Spurs,Forward,5.650000,3.700000,0.033046
65,Southampton,Forward,5.366667,1.666667,0.026963
1,Arsenal,Forward,5.200000,2.300000,0.029183
25,Crystal Palace,Forward,5.050000,1.300000,0.024583
5,Aston Villa,Forward,4.966667,2.533333,0.038538
45,Liverpool,Forward,4.766667,3.533333,0.040617
41,Leicester,Forward,4.733333,1.433333,0.022191


# Value

In [70]:
df.sort_values('value', ascending=False).head(10)

,id,web_name,team,selected_by_percent,now_cost,minutes,total_points,form,goals_scored,assists,...,form_rank_type,points_per_game_rank,points_per_game_rank_type,selected_rank,selected_rank_type,starts_per_90,clean_sheets_per_90,position,value,form_per_cost
478,357,Trippier,Newcastle,64.9,59,1324,96,10.0,1,4,...,1,5,1,2,1,0.06798,0.61178,Defender,16.3,0.169492
490,369,Almirón,Newcastle,35.2,58,1255,93,8.7,8,2,...,4,7,3,7,3,0.07171,0.57371,Midfielder,16.0,0.150000
312,254,Ward,Leicester,28.4,41,1350,60,6.7,0,0,...,1,62,10,10,1,0.06667,0.40000,Goalkeeper,14.6,0.163415
523,398,Henderson,Nott'm Forest,12.5,47,1350,67,2.3,0,0,...,15,41,5,31,6,0.06667,0.26667,Goalkeeper,14.3,0.048936
23,26,Saliba,Arsenal,35.1,53,1245,74,7.0,2,1,...,3,18,2,8,3,0.07229,0.50602,Defender,14.0,0.132075
8,10,White,Arsenal,10.6,47,1150,66,6.3,0,2,...,4,31,7,39,12,0.07826,0.62609,Defender,14.0,0.134043
316,258,Castagne,Leicester,9.8,47,1191,65,4.7,1,2,...,14,37,9,40,13,0.07557,0.52897,Defender,13.8,0.100000
14,16,Gabriel,Arsenal,13.5,52,1260,72,8.7,2,0,...,2,21,4,29,8,0.07143,0.50000,Defender,13.8,0.167308
487,366,Schär,Newcastle,11.6,49,1244,67,5.0,1,0,...,10,30,6,35,11,0.07235,0.57878,Defender,13.7,0.102041
293,346,Andreas,Fulham,23.7,46,1240,61,3.7,2,5,...,32,55,26,13,6,0.07258,0.21774,Midfielder,13.3,0.080435


In [ ]:
pivot=df.pivot_table(index='position',values='value',aggfunc=np.mean).reset_index()
pivot.sort_values('value',ascending=False)

In [ ]:
fwd_df = df.loc[df.position == 'Forward']
mid_df = df.loc[df.position == 'Midfielder']
def_df = df.loc[df.position == 'Defender']
goal_df = df.loc[df.position == 'Goalkeeper']

In [ ]:
goal_df.value.hist()

In [ ]:
goal_df.sort_values('value',ascending=False).head(10)

In [ ]:
def_df.value.hist()

In [ ]:
def_df.sort_values('value',ascending=False).head(10)
def_df.loc[def_df.minutes > 1000,:].sort_values('value',ascending=True).head(10)


In [ ]:
mid_df.value.hist()

In [ ]:
mid_df.sort_values('value', ascending=False).head(10)

In [ ]:
fwd_df.value.hist()

In [ ]:
fwd_df.sort_values('value', ascending=False).head(10)

# Form

In [ ]:
df.sort_values('form', ascending=False).head(30)

In [ ]:
df.sort_values('form_per_cost', ascending=False).head(30)

In [ ]:
pivot=df.pivot_table(index='position',values=['form', 'form_per_cost'],aggfunc=np.mean).reset_index()
pivot.sort_values('form',ascending=False)

In [ ]:
pivot.sort_values('form_per_cost',ascending=False)

In [ ]:
goal_df.form_per_cost.hist()

In [ ]:
goal_df.sort_values('form_per_cost', ascending=False).head(10)

In [ ]:
def_df.form_per_cost.hist()

In [ ]:
def_df.sort_values('form_per_cost', ascending=False).head(15)

In [ ]:
mid_df.form_per_cost.hist()

In [ ]:
mid_df.sort_values('form_per_cost', ascending=False).head(15)

In [ ]:
fwd_df.form_per_cost.hist()

In [ ]:
fwd_df.sort_values('form_per_cost', ascending=False).head(15)

In [74]:
team_pivot.team.value_counts()

AttributeError: 'Series' object has no attribute 'count_values'